In [2]:
import torch
from torch import nn
import numpy as np
import cv2

In [5]:
class AlexNet(nn.Module) :
    def __init__(self, 
                 first_in=3, 
                 first_stride=4, 
                 dropout_rate=0.5, 
                 in_feature=256*6*6,
                 out_feature=4608,
                 num_classes=1000
                ) :
        super(AlexNet, self).__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv2d(in_channels=first_in, 
                      out_channels=96, 
                      kernel_size=11, 
                      padding=2, 
                      stride=first_stride),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, 
                                 k=2,    
                                 alpha=0.0001, 
                                 beta=0.75),
            nn.MaxPool2d(kernel_size=3, 
                        stride=2),
        )
        
        self.conv_2 = nn.Sequential(
            nn.Conv2d(in_channels=96, 
                     out_channels=256,
                      padding=2,
                      kernel_size=5),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, 
                                 k=2,    
                                 alpha=0.0001, 
                                 beta=0.75),
            nn.MaxPool2d(kernel_size=3, 
                        stride=2),
        )
        
        self.conv_3 = nn.Sequential(
            nn.Conv2d(in_channels=256, 
                     out_channels=384,
                      padding=1,
                      kernel_size=3),
            nn.ReLU(),
        )
        
        self.conv_4 = nn.Sequential(
            nn.Conv2d(in_channels=384, 
                     out_channels=384,
                      padding=1,
                      kernel_size=3),
            nn.ReLU(),
        )
        
        self.conv_5 = nn.Sequential(
            nn.Conv2d(in_channels=384, 
                     out_channels=256,
                      padding=1,
                      kernel_size=3,
                     ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, 
                         stride=2),
        )
                
        self.conv_layers = nn.Sequential(
            self.conv_1, 
            self.conv_2, 
            self.conv_3, 
            self.conv_4, 
            self.conv_5, 
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(in_feature, out_feature),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(out_feature, out_feature),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(out_feature, num_classes),
        )

        
        for idx, layers in enumerate(self.conv_layers):
            self.initialization(idx, layers)

        self.initialization('fc', self.fc_layers)

    def initialization(self, idx, layers) :
        for layer in layers :
            if 'weight' in dir(layer):
                nn.init.normal_(layer.weight, mean=0, std=0.01)                
                if idx in ['fc', 1, 3, 4] :
                    nn.init.constant_(layer.bias.data, 1)
                elif idx in [0, 2] :
                    nn.init.constant_(layer.bias.data, 0)    
                    
    def forward(self, x) :
        x = self.conv_layers(x)
        # x = x.contiguous().view(-1) 

        x = x.flatten(1)

        x = self.fc_layers(x)
        return x
    


In [6]:
test_data = torch.randn(16, 27, 75, 75)
model = AlexNet(27,2, in_feature=256*3*3)
out = model(test_data)

torch.Size([16, 256, 3, 3])
torch.Size([16, 2304])


c:\users\quhb2\anaconda3\envs\torch-1.9\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
model.fc_layers[0].bias

Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], requires_grad=True)

In [9]:
class CFN(nn.Module) :
    def __init__(self, 
                 in_channel=27, 
                 strd=2, 
                 in_feature=256*3*3, 
                 out_feature=4608, 
                 num_classes=69) :
        super(CFN, self).__init__()
        self.alexnet = AlexNet(first_in=in_channel, 
                               first_stride=strd, 
                               in_feature=in_feature, 
                               out_feature=out_feature
                              )
        # alexnet 논문에 나와있는 방법으로 초기화한 layer들을 가져옴
        self.conv_layers = self.alexnet.conv_layers
        self.fc6 = self.alexnet.fc_layers[0]
        
        # fc7, fc8, output 포함
        self.classifier = nn.Sequential(
            nn.Linear(out_feature, 4096), 
            nn.ReLU(),
            nn.Linear(4096, 100), 
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )
        
        self.alexnet.initialization('fc', self.classifier)
        
    def forward(self, x) :
        x = self.conv_layers(x)
        x = x.flatten(1)
        x = self.fc6(x)
        x = self.classifier(x)
        return x

In [10]:
test_data = torch.randn(16, 27, 75, 75)
model = CFN()
out = model(test_data)
print(out.shape)

torch.Size([16, 69])


In [11]:
model.classifier[0].bias

Parameter containing:
tensor([1., 1., 1.,  ..., 1., 1., 1.], requires_grad=True)